In [1]:
import bs4 as bs
import urllib.request
import pandas as pd
#Open the cat web data page
cat_data = urllib.request.urlopen('https://simple.wikipedia.org/wiki/Cat').read()

# Retrieve the questions and answers
tweet_data = pd.read_csv("question_responce.csv")
tweet_data = tweet_data[["question","responce"]]
#Find all the paragraph html from the web page
cat_data_paragraphs  = bs.BeautifulSoup(cat_data,'lxml').find_all('p')
#Creating the corpus of all the web page paragraphs
cat_text = ''
#Creating lower text corpus of cat paragraphs
for p in cat_data_paragraphs:
    cat_text += p.text.lower()


In [2]:
tweet_data.head()

,question,responce
0,@AmericanAir Erica on the lax team is amazing ...,@115904 We'll be sure to pass along your kind ...
1,@AmericanAir Could you have someone on your la...,@115904 Our apologies for the delay in respond...
2,Ben Tennyson and an American Airlines pilot. 🎃...,"@115905 Aww, that's definitely a future pilot ..."
3,"@AmericanAir Right, but I earned those. I also...",@115906 We're sorry for your frustration.
4,"Thank you, @AmericanAir for playing #ThisIsUs ...",@115909 We're glad you got to kick back and en...


In [3]:
import re
cat_text = re.sub(r'\s+', ' ',re.sub(r'\[[0-9]*\]', ' ', cat_text))


In [5]:
import nltk
nltk.download('punkt')
cat_sentences = nltk.sent_tokenize(cat_text)

[nltk_data] Downloading package punkt to /home/julie/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
def chatbot_answer(user_query):
    
    #Append the query to the sentences list
    cat_sentences.append(user_query)
    #Create the sentences vector based on the list
    vectorizer = TfidfVectorizer()
    sentences_vectors = vectorizer.fit_transform(cat_sentences)
    
    #Measure the cosine similarity and take the second closest index because the first index is the user query
    vector_values = cosine_similarity(sentences_vectors[-1], sentences_vectors)
    answer = cat_sentences[vector_values.argsort()[0][-2]]
    #Final check to make sure there are result present. If all the result are 0, means the text input by us are not captured in the corpus
    input_check = vector_values.flatten()
    input_check.sort()
    
    if input_check[-2] == 0:
        return "Please Try again"
    else: 
        return answer

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
print("Hello, I am the Cat Chatbot. What is your meow questions?:")
while(True):
    query = input().lower()
    if query not in ['bye', 'good bye', 'take care']:
        print("Cat Chatbot: ", end="")
        print(chatbot_answer(query))
        cat_sentences.remove(query)
    else:
        print("See You Again")
        break

Hello, I am the Cat Chatbot. What is your meow questions?:


KeyboardInterrupt: ignored